In [17]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import os, sys

In [19]:
import torch
import torch.nn as nn

In [20]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import random

## Siamese networks

In [21]:
trainset = datasets.ImageFolder('./omniglot_data/changed/train')
valset = datasets.ImageFolder('./omniglot_data/changed/valid')
testset =  datasets.ImageFolder('./omniglot_data/changed/test')

In [22]:
random.choice(trainset.imgs)

('./omniglot_data/changed/train/Balinese_character07/0114_11.png', 101)

In [38]:
# from dataloader import SiameseTestData, SiameseTrainData
from networks import SiameseNet
from losses import ContrastiveLoss
import torchvision.models as models
import torch.optim.lr_scheduler as lr_scheduler
from utils import AverageMeter
import time

In [28]:
input_size = 105
learning_rate = 1e3
epochs = 200
batch_size = 32
num_workers = 4

In [29]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    cuda = True
else:
    device = torch.device("cpu")
device

device(type='cuda')

In [30]:
data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ]),
        'val': transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    }

In [31]:
from torch.utils.data.sampler import SubsetRandomSampler

In [32]:
omniglot_trainset = datasets.Omniglot(root='./omniglot_data/', download=True, background=True, transform=data_transforms['train'])
omniglot_valset = datasets.Omniglot(root='./omniglot_data/', download=True, background=True, transform=data_transforms['val'])
omniglot_evalset = datasets.Omniglot(root='./omniglot_data/', download=True, background=False, transform=data_transforms['val'])

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [33]:
indices = list(range(len(omniglot_trainset)))
split = int(0.15 * len(omniglot_trainset))
train_indices = indices[:split]
val_indices = indices[split:]
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

trainset = SiameseTrainData(omniglot_trainset)
valset = SiameseTestData(omniglot_valset)
testset = SiameseTestData(omniglot_evalset)

NameError: name 'SiameseTrainData' is not defined

In [34]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, sampler=train_sampler, num_workers=4, pin_memory=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, sampler=val_sampler, num_workers=4, pin_memory=True)

In [35]:
class KochNet(nn.Module):

    def __init__(self):
        super(KochNet, self).__init__()
        self.features = nn.Sequential(
            # 1x105x105
            nn.Conv2d(1, 64, kernel_size=10),
            # 64x96x96
            nn.ReLU(inplace=True), 
            nn.MaxPool2d(kernel_size=2),
            # 64x48x48
            nn.Conv2d(64, 128, kernel_size=7),
            # 128x42x42
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            # 128x21x21
            nn.Conv2d(128, 128, kernel_size=4),
            # 128x18x18
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            # 128x9x9
            nn.Conv2d(128, 256, kernel_size=4),
            # 256x6x6
            nn.ReLU(inplace=True),
        )
        self.fc = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.Sigmoid(),
        )
        self.output = nn.Linear(4096, 1)
        

    def forward(self, x1, x2):
        x1 = self.features(x1)
        x1 = x1.view(x1.size(0), 256 * 6 * 6)
        x1 = self.fc(x1)
        x2 = self.features(x2)
        x2 = x2.view(x2.size(0), 256 * 6 * 6)
        x2 = self.fc(x2)
        dist = torch.abs(x1 - x2)
        out = self.output(dist)
        return out

In [39]:
model = KochNet()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
T_max = epochs
eta_min = 0.01
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max, eta_min=eta_min)
criterion = nn.BCEWithLogitsLoss()
if cuda:
    model.to(device)

In [15]:
def train(train_loader, model, criterion, optimizer, epoch, device, debug=False, print_freq=200):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for batch_idx, (imgs1, imgs2, targets) in enumerate(train_loader):
        data_time.update(time.time() - end)

        imgs1 = imgs1.to(device).float()
        imgs2 = imgs2.to(device).float()
        targets = targets.to(device).float()

        output = model(imgs1, imgs2)

        loss = criterion(output, targets)

        losses.update(loss.item(), imgs1.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if batch_idx % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})'.format(
                      epoch, batch_idx, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses))
        if debug:
            break

    return losses.avg

In [ ]:
train(trainloader, model, criterion, optimizer, 1, device, False, 1)

Epoch: [1][0/91]	Time 45.901 (45.901)	Data 45.784 (45.784)	Loss 0.6955 (0.6955)
Epoch: [1][1/91]	Time 0.226 (23.064)	Data 0.001 (22.892)	Loss 263082.5938 (131541.6446)
Epoch: [1][2/91]	Time 0.217 (15.448)	Data 0.001 (15.262)	Loss 93686.3047 (118923.1980)
Epoch: [1][3/91]	Time 0.194 (11.634)	Data 0.000 (11.446)	Loss 284424.8125 (160298.6016)
Epoch: [1][4/91]	Time 73.469 (24.001)	Data 73.384 (23.834)	Loss 95561.3828 (147351.1579)
Epoch: [1][5/91]	Time 0.236 (20.040)	Data 0.001 (19.862)	Loss 67419.9219 (134029.2852)
Epoch: [1][6/91]	Time 0.210 (17.207)	Data 0.008 (17.025)	Loss 52753.2773 (122418.4269)
Epoch: [1][7/91]	Time 0.211 (15.083)	Data 0.001 (14.897)	Loss 46456.9648 (112923.2442)
Epoch: [1][8/91]	Time 39.373 (17.782)	Data 39.278 (17.606)	Loss 81413.7266 (109422.1867)
Epoch: [1][9/91]	Time 8.915 (16.895)	Data 8.814 (16.727)	Loss 35393.8477 (102019.3528)
Epoch: [1][10/91]	Time 0.238 (15.381)	Data 0.001 (15.207)	Loss 71496.3984 (99244.5387)
Epoch: [1][11/91]	Time 0.219 (14.117)	Data 0

In [43]:
for i in range(10):
    scheduler.step()

In [44]:
scheduler.last_epoch

10

In [45]:
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max, eta_min=eta_min)



In [46]:
scheduler.last_epoch

-1